<a href="https://colab.research.google.com/github/Priyo-prog/Deep-Learning-with-Tensorflow/blob/main/Neural%20Network%20Regression%20with%20Tensorflow/Neural_Network_Medical_Cost_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
# Read the insuarnce dataset
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
# Let's try one hot encode using pandas get.dummies()
insurance_one_hot = pd.get_dummies(insurance)
insurance_one_hot.head(10)

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
5,31,25.740,0,3756.62160,1,0,1,0,0,0,1,0
6,46,33.440,1,8240.58960,1,0,1,0,0,0,1,0
7,37,27.740,3,7281.50560,1,0,1,0,0,1,0,0
8,37,29.830,2,6406.41070,0,1,1,0,1,0,0,0
9,60,25.840,0,28923.13692,1,0,1,0,0,1,0,0


In [10]:
# Create X & y (features and labels)
X = insurance_one_hot.drop("charges", axis=1)

y = insurance_one_hot["charges"]
X.head(5), y.head(5)

(   age     bmi  children  ...  region_northwest  region_southeast  region_southwest
 0   19  27.900         0  ...                 0                 0                 1
 1   18  33.770         1  ...                 0                 1                 0
 2   28  33.000         3  ...                 0                 1                 0
 3   33  22.705         0  ...                 1                 0                 0
 4   32  28.880         0  ...                 1                 0                 0
 
 [5 rows x 11 columns], 0    16884.92400
 1     1725.55230
 2     4449.46200
 3    21984.47061
 4     3866.85520
 Name: charges, dtype: float64)

In [13]:
# Create training and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
len(X), len(X_train), len(X_test)

(1338, 1070, 268)

In [14]:
# Create a neural network model

# Create a model
insurance_model = tf.keras.Sequential([
                 tf.keras.layers.Dense(10),
                 tf.keras.layers.Dense(1)                      
])


# 2. Compile the model
insurance_model.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.SGD(),
                        metrics=["mae"])

# 3. Fit the model
insurance_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
34/34 [==============================] - 1s 1ms/step - loss: 8437.4932 - mae: 8437.4932
Epoch 2/100
34/34 [==============================] - 0s 1ms/step - loss: 7689.9946 - mae: 7689.9946
Epoch 3/100
34/34 [==============================] - 0s 1ms/step - loss: 7491.8760 - mae: 7491.8760
Epoch 4/100
34/34 [==============================] - 0s 1ms/step - loss: 7725.5767 - mae: 7725.5767
Epoch 5/100
34/34 [==============================] - 0s 1ms/step - loss: 7734.2046 - mae: 7734.2046
Epoch 6/100
34/34 [==============================] - 0s 1ms/step - loss: 7806.8047 - mae: 7806.8047
Epoch 7/100
34/34 [==============================] - 0s 1ms/step - loss: 7661.0723 - mae: 7661.0723
Epoch 8/100
34/34 [==============================] - 0s 1ms/step - loss: 7611.1021 - mae: 7611.1021
Epoch 9/100
34/34 [==============================] - 0s 1ms/step - loss: 7712.8838 - mae: 7712.8838
Epoch 10/100
34/34 [==============================] - 0s 1ms/step - loss: 7614.1284 - mae: 7614.1284

In [15]:
# Check the results of the insurance_model on the test data
insurance_model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 5848.1196 - mae: 5848.1196


[5848.11962890625, 5848.11962890625]

In [16]:
y_train.median(), y_train.mean() # Model not performing well

(9575.4421, 13346.089736364489)